# Julia DSL Example: Ag-Cu FCC Phase

This notebook demonstrates how to define thermodynamic models using Julia functions instead of TDB files.

**Advantages of Julia DSL:**
- No TDB file parsing needed
- Automatic differentiation works through the entire calculation
- Easy to modify and experiment with parameters

In [ ]:
using OpenCALPHAD
using ForwardDiff

## 1. Define Phase using Julia DSL

Create an FCC phase with Ag and Cu on a single sublattice.

In [ ]:
# Create FCC phase with single sublattice containing Ag and Cu
fcc = Phase("FCC_A1", [1.0], [[:AG, :CU]])

# Set Gibbs energy functions for pure elements (GHSER)
set_G!(fcc, [:AG], T -> -7209.512 + 118.202 * T - 23.8463 * T * log(T))
set_G!(fcc, [:CU], T -> -7770.458 + 130.485 * T - 24.1124 * T * log(T))

# Set interaction parameters (Redlich-Kister L0 and L1)
set_L!(fcc, [:AG, :CU], 0, T -> 33819.09 - 8.1236 * T)
set_L!(fcc, [:AG, :CU], 1, T -> -5601.87 + 1.32997 * T)

println("Phase: $(fcc.name)")
println("Constituents: $(fcc.constituents)")

## 2. Calculate Gibbs Energy

In [ ]:
T = 1000.0  # Temperature [K]
x_Cu = 0.3  # Mole fraction of Cu

# Create site fraction matrix
y = zeros(1, 2)
y[1, 1] = 1.0 - x_Cu  # x_Ag
y[1, 2] = x_Cu        # x_Cu

G = calculate_gibbs_energy(fcc, T, y)
println("At T = $(T) K, x(Cu) = $(x_Cu):")
println("  Gibbs energy: $(round(G, digits=2)) J/mol")

## 3. Find Miscibility Gap

In [ ]:
gap = find_miscibility_gap(fcc, T)

println("Miscibility gap at T = $(T) K:")
println("  Ag-rich phase: x(Cu) = $(round(1-gap.x2, digits=4))")
println("  Cu-rich phase: x(Cu) = $(round(1-gap.x1, digits=4))")

## 4. Temperature Scan

In [ ]:
println("  T [K]  |  Ag-rich x(Cu)  |  Cu-rich x(Cu)")
println("-" ^ 50)

for T_scan in [800.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0]
    gap_T = find_miscibility_gap(fcc, T_scan)
    if !isnothing(gap_T)
        x_Cu_Ag_rich = 1.0 - gap_T.x2
        x_Cu_Cu_rich = 1.0 - gap_T.x1
        println("  $(Int(T_scan))   |     $(round(x_Cu_Ag_rich, digits=4))      |     $(round(x_Cu_Cu_rich, digits=4))")
    end
end

## 5. Automatic Differentiation

The key advantage of Julia DSL: ForwardDiff.jl can differentiate through the entire calculation.

In [ ]:
# Define Gibbs energy as a function of composition
function G_of_x(x_Cu_val)
    y_ad = zeros(eltype(x_Cu_val), 1, 2)
    y_ad[1, 1] = 1.0 - x_Cu_val
    y_ad[1, 2] = x_Cu_val
    return calculate_gibbs_energy(fcc, 1000.0, y_ad)
end

# Compute derivative dG/dx using ForwardDiff
x_test = 0.3
dG_dx = ForwardDiff.derivative(G_of_x, x_test)

println("At T = 1000 K, x(Cu) = $(x_test):")
println("  G = $(round(G_of_x(x_test), digits=2)) J/mol")
println("  dG/dx(Cu) = $(round(dG_dx, digits=2)) J/mol")

## 6. Visualize Gibbs Energy Curve

In [ ]:
using Plots

x_range = 0.01:0.01:0.99
G_values = [G_of_x(x) for x in x_range]

plot(x_range, G_values,
    xlabel="x(Cu)",
    ylabel="Gibbs Energy [J/mol]",
    title="Ag-Cu FCC at T = 1000 K (Julia DSL)",
    legend=false,
    linewidth=2)